In [1]:
import pandas as pd
import numpy as np
import nibabel as nib
from copy import copy
import logging
from utils import mult_diag, counter
from multiprocessing import Pool
import pickle

%pylab inline

Populating the interactive namespace from numpy and matplotlib


/usr/local/lib/python2.7/dist-packages/IPython/core/magics/pylab.py:161: UserWarning: pylab import has clobbered these variables: ['copy']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [2]:
with open("srt/stimulus.pcl", 'rb') as f:
    stimulus = pickle.load(f)

In [10]:
mri = np.load("fmri/mri_all1.npy")

In [6]:
logger = logging.getLogger('bu')

In [3]:
zs = lambda v: (v-v.mean(0))/v.std(0) ## z-score function

def ridge_corr(Rstim, Pstim, Rresp, Presp, alphas, normalpha=False, dtype=np.single, corrmin=0.2,
               singcutoff=1e-10, use_corr=True, logger=logging.getLogger("ridge_corr")):
    """Uses ridge regression to find a linear transformation of [Rstim] that approximates [Rresp].
    Then tests by comparing the transformation of [Pstim] to [Presp]. This procedure is repeated
    for each regularization parameter alpha in [alphas]. The correlation between each prediction and
    each response for each alpha is returned. Note that the regression weights are NOT returned.

    Parameters
    ----------
    Rstim : array_like, shape (TR, N)
        Training stimuli with TR time points and N features. Each feature should be Z-scored across time.
    Pstim : array_like, shape (TP, N)
        Test stimuli with TP time points and N features. Each feature should be Z-scored across time.
    Rresp : array_like, shape (TR, M)
        Training responses with TR time points and M responses (voxels, neurons, what-have-you).
        Each response should be Z-scored across time.
    Presp : array_like, shape (TP, M)
        Test responses with TP time points and M responses.
    alphas : list or array_like, shape (A,)
        Ridge parameters to be tested. Should probably be log-spaced. np.logspace(0, 3, 20) works well.
    normalpha : boolean
        Whether ridge parameters should be normalized by the Frobenius norm of Rstim. Good for
        comparing models with different numbers of parameters.
    dtype : np.dtype
        All data will be cast as this dtype for computation. np.single is used by default for memory
        efficiency.
    corrmin : float in [0..1]
        Purely for display purposes. After each alpha is tested, the number of responses with correlation
        greater than corrmin minus the number of responses with correlation less than negative corrmin
        will be printed. For long-running regressions this vague metric of non-centered skewness can
        give you a rough sense of how well the model is working before it's done.
    singcutoff : float
        The first step in ridge regression is computing the singular value decomposition (SVD) of the
        stimulus Rstim. If Rstim is not full rank, some singular values will be approximately equal
        to zero and the corresponding singular vectors will be noise. These singular values/vectors
        should be removed both for speed (the fewer multiplications the better!) and accuracy. Any
        singular values less than singcutoff will be removed.
    use_corr : boolean
        If True, this function will use correlation as its metric of model fit. If False, this function
        will instead use variance explained (R-squared) as its metric of model fit. For ridge regression
        this can make a big difference -- highly regularized solutions will have very small norms and
        will thus explain very little variance while still leading to high correlations, as correlation
        is scale-free while R**2 is not.

    Returns
    -------
    Rcorrs : array_like, shape (A, M)
        The correlation between each predicted response and each column of Presp for each alpha.
    
    """
    ## Calculate SVD of stimulus matrix
    #logging.basicConfig(filename='example.log',level=logging.DEBUG)
    
    logger.info("Doing SVD...")
    try:
        U,S,Vh = np.linalg.svd(Rstim, full_matrices=False)
    except np.linalg.LinAlgError, e:
        logger.info("NORMAL SVD FAILED, trying more robust dgesvd..")
        from text.regression.svd_dgesvd import svd_dgesvd
        U,S,Vh = svd_dgesvd(Rstim, full_matrices=False)

    ## Truncate tiny singular values for speed
    origsize = S.shape[0]
    ngoodS = np.sum(S>singcutoff)
    nbad = origsize-ngoodS
    U = U[:,:ngoodS]
    S = S[:ngoodS]
    Vh = Vh[:ngoodS]
    logger.info("Dropped %d tiny singular values.. (U is now %s)"%(nbad, str(U.shape)))

    ## Normalize alpha by the Frobenius norm
    #frob = np.sqrt((S**2).sum()) ## Frobenius!
    frob = S[0]
    #frob = S.sum()
    logger.info("Training stimulus has Frobenius norm: %0.03f"%frob)
    if normalpha:
        nalphas = alphas * frob
    else:
        nalphas = alphas

    ## Precompute some products for speed
    UR = np.dot(U.T, Rresp) ## Precompute this matrix product for speed
    PVh = np.dot(Pstim, Vh.T) ## Precompute this matrix product for speed
    
    #Prespnorms = np.apply_along_axis(np.linalg.norm, 0, Presp) ## Precompute test response norms
    zPresp = zs(Presp)
    Prespvar = Presp.var(0)
    Rcorrs = [] ## Holds training correlations for each alpha
    for na, a in zip(nalphas, alphas):
        #D = np.diag(S/(S**2+a**2)) ## Reweight singular vectors by the ridge parameter 
        D = S/(S**2+na**2) ## Reweight singular vectors by the (normalized?) ridge parameter
        
        pred = np.dot(mult_diag(D, PVh, left=False), UR) ## Best (1.75 seconds to prediction in test)
        # pred = np.dot(mult_diag(D, np.dot(Pstim, Vh.T), left=False), UR) ## Better (2.0 seconds to prediction in test)
        
        # pvhd = reduce(np.dot, [Pstim, Vh.T, D]) ## Pretty good (2.4 seconds to prediction in test)
        # pred = np.dot(pvhd, UR)
        
        # wt = reduce(np.dot, [Vh.T, D, UR]).astype(dtype) ## Bad (14.2 seconds to prediction in test)
        # wt = reduce(np.dot, [Vh.T, D, U.T, Rresp]).astype(dtype) ## Worst
        # pred = np.dot(Pstim, wt) ## Predict test responses

        if use_corr:
            #prednorms = np.apply_along_axis(np.linalg.norm, 0, pred) ## Compute predicted test response norms
            #Rcorr = np.array([np.corrcoef(Presp[:,ii], pred[:,ii].ravel())[0,1] for ii in range(Presp.shape[1])]) ## Slowly compute correlations
            #Rcorr = np.array(np.sum(np.multiply(Presp, pred), 0)).squeeze()/(prednorms*Prespnorms) ## Efficiently compute correlations
            Rcorr = (zPresp*zs(pred)).mean(0)
        else:
            ## Compute variance explained
            resvar = (Presp-pred).var(0)
            Rcorr = np.clip(1-(resvar/Prespvar), 0, 1)
            
        #Rcorr[np.isnan(Rcorr)] = 0.
        Rcorrs.append(Rcorr)
        
        log_template = "Training: alpha=%0.3f, mean corr=%0.5f, max corr=%0.5f, over-under(%0.2f)=%d"
        log_msg = log_template % (a,
                                  np.mean(Rcorr),
                                  np.max(Rcorr),
                                  corrmin,
                                  (Rcorr>corrmin).sum()-(-Rcorr>corrmin).sum())
        if logger is not None:
            logger.info(log_msg)
        else:
            print log_msg
    
    return Rcorrs

In [21]:
coordinates = [(x, y, z) for x in range(20, 160, 3) for y in range(20, 160, 3) for z in range(4, 36, 4)]

In [22]:
def do_ridge(triple):
    print "processing ", triple
    try:
        res = ridge_corr(stimulus[:-500], stimulus[-500:],
                         mri[triple[0], triple[1], triple[2]][:-500],
                         mri[triple[0], triple[1], triple[2]][-500:],
                         range(30, 200, 10))
    except Exception as e:
        print e

    return res

In [ ]:
p = Pool(16)
result = p.map(do_ridge, coordinates)

processing  (20, 20, 4)
processing  (20, 122, 24)
processing  (32, 77, 28)
processing  (26, 47, 32)
processing  (26, 152, 20)
processing  (35, 41, 16)
processing  (38, 107, 24)
processing  (29, 116, 8)
processing  (35, 146, 4)
processing  (44, 32, 32)
processing  (47, 101, 8)
processing  (50, 62, 28)
processing  (44, 137, 20)
processing  (53, 26, 16)
processing  (23, 86, 12)
processing  (41, 71, 12)
processing  (50, 62, 32)
processing  (23, 86, 16)
processing  (29, 116, 12)
processing  (35, 41, 20)
processing  (53, 26, 20)
processing  (41, 71, 16)
processing  (26, 50, 4)
processing  (32, 77, 32)
processing  (47, 101, 12)
processing  (20, 20, 8)
processing  (44, 35, 4)
processing  (26, 152, 24)
processing  (35, 146, 8)
processing  (20, 122, 28)
processing  (38, 107, 28)
processing  (44, 137, 24)
processing  (41, 71, 20)
processing  (29, 116, 16)
processing  (20, 20, 12)
processing  (35, 41, 24)
processing  (32, 80, 4)
processing  (47, 101, 16)
processing  (23, 86, 20)
processing  (26, 5

In [ ]:
result = np.array(result)

In [ ]:
result_coord = zip(coordinates, result)

In [ ]:
for i, alpha in enumerate(range(30, 200, 10)):
    title("alpha = {}".format(alpha))
    hist(result[:, i])
    show()

In [ ]:
np.save("result/results2", result_coord)

In [ ]:
print("ok")